In [39]:
! pip install transformers datasets evaluate

In [31]:
!pip install huggingface_hub
from huggingface_hub import notebook_login
notebook_login()

In [40]:
from datasets import load_dataset

squad = load_dataset("squad", split="train[:5000]")

In [41]:
squad = squad.train_test_split(test_size=0.2)

In [42]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [43]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [44]:
tokenized_squad = squad.map(preprocess_function, batched=True, remove_columns=squad["train"].column_names)

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [45]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [46]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator(return_tensors="tf")

In [47]:
from transformers import create_optimizer

batch_size = 16
num_epochs = 2
total_train_steps = (len(tokenized_squad["train"]) // batch_size) * num_epochs
optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=0,
    num_train_steps=total_train_steps,)

In [48]:
from transformers import TFAutoModelForQuestionAnswering, AutoTokenizer

# Replace 'model_name_or_path' with the name of the pretrained model you want to use
model_name_or_path = "distilbert-base-uncased"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

# Load the model
model = TFAutoModelForQuestionAnswering.from_pretrained(model_name_or_path)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForQuestionAnswering: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_transform.weight']
- This IS expected if you are initializing TFDistilBertForQuestionAnswering from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForQuestionAnswering from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForQuestionAnswering were not initialized from the PyTorch model and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it

In [49]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_squad["train"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    tokenized_squad["test"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

In [56]:
import tensorflow as tf

model.compile(optimizer=optimizer)
!pip install huggingface_hub
from huggingface_hub import notebook_login
notebook_login()

from transformers.keras_callbacks import PushToHubCallback

callback = PushToHubCallback(
    output_dir="my_awesome_qa_model",
    tokenizer=tokenizer,
)

/content/my_awesome_qa_model is already a clone of https://huggingface.co/Madhur-01/my_awesome_qa_model. Make sure you pull the latest changes with `repo.git_pull()`.


In [57]:
model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=3, callbacks=[callback])

Epoch 1/3
250/250 [==============================] - ETA: 0s - loss: 3.4245

Several commits (4) will be pushed upstream.


250/250 [==============================] - 264s 921ms/step - loss: 3.4245 - val_loss: 2.1038
Epoch 2/3
250/250 [==============================] - ETA: 0s - loss: 1.7543

Several commits (5) will be pushed upstream.


250/250 [==============================] - 199s 796ms/step - loss: 1.7543 - val_loss: 1.7800
Epoch 3/3
250/250 [==============================] - ETA: 0s - loss: 1.4973

Several commits (6) will be pushed upstream.


250/250 [==============================] - 189s 754ms/step - loss: 1.4973 - val_loss: 1.7800


In [59]:
model.push_to_hub("Madhur-01/my_awesome_qa_model")
tokenizer.push_to_hub("Madhur-01/my_awesome_qa_model")

CommitInfo(commit_url='https://huggingface.co/Madhur-01/my_awesome_qa_model/commit/0911e82edb16cae5714fd829fff331c3b084a011', commit_message='Upload tokenizer', commit_description='', oid='0911e82edb16cae5714fd829fff331c3b084a011', pr_url=None, pr_revision=None, pr_num=None)